In [ ]:
from glob import glob
from pathlib import Path

# Discover available tables from layer_01_bronze
paths = glob('../layer_01_bronze/*.json')
table_map = {Path(p).stem: p for p in paths}
tables = sorted(table_map.keys())

dbutils.widgets.combobox('table', tables[0] if tables else '', tables)

In [ ]:
import json

table_name = dbutils.widgets.get('table')
settings_path = table_map[table_name]
with open(settings_path) as f:
    job_settings = json.load(f)

full_table_name = job_settings['dst_table_name']
history_schema = job_settings.get('history_schema')
catalog = full_table_name.split('.')[0]

In [ ]:
from functions.history import build_and_merge_file_history, transaction_history
from functions.utility import schema_exists

settings_message = '

Dictionary dynamically generated from JSON file:

'
settings_message += json.dumps({'full_table_name': full_table_name, 'history_schema': history_schema}, indent=4)
print(settings_message)

if history_schema is None:
    print('Skipping history build: no history_schema provided')
elif schema_exists(catalog, history_schema, spark):
    build_and_merge_file_history(full_table_name, history_schema, spark)
    transaction_history(full_table_name, history_schema, spark)
else:
    print(f'Skipping history build: schema {catalog}.{history_schema} not found')